In [1]:
import json
from pathlib import Path

import keras
from keras import Sequential, Input
import keras_tuner
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Dropout, Dense
from keras.metrics import RootMeanSquaredError
from keras.src.optimizers import Adam

from utils.centralized_training import _build_centralized_model_dataset
from gossiplearning.config import Config
from keras.src.layers import RepeatVector

In [4]:
with open("azurefunctions_config.json", "r") as f:
  config = Config.model_validate(json.load(f))

INPUT_SHAPE = (
  config.training.input_timesteps, config.training.n_input_features
)

In [13]:
def model_builder(hp: keras_tuner.HyperParameters) -> keras.Model:
    model = Sequential()
    # LSTM layer
    model.add(
        LSTM(
            hp.Int(
                min_value=16, max_value=320, step=16, name="lstm_layer_1_dim"
            ),
            activation="tanh",
            input_shape=INPUT_SHAPE,
            return_sequences=True,
        )
    )
    # dropout
    if hp.Boolean("dropout_1"):
        model.add(
            Dropout(
                hp.Float(
                    name="dropout_1_rate", 
                    min_value=0.1, 
                    max_value=0.5, 
                    step=0.05
                )
            )
        )
    # LSTM layer
    if hp.Boolean("lstm_2"):
        # model.add(RepeatVector(1)),
        model.add(
            LSTM(
                hp.Int(
                    min_value=16, 
                    max_value=320, 
                    step=16, 
                    name="lstm_layer_2_dim"
                ),
                activation="tanh",
                input_shape=(4, 14),
                # return_sequences=True,
            )
        )
    # dropout
    if hp.Boolean("dropout_2"):
        model.add(
            Dropout(
                hp.Float(
                    name="dropout_2_rate", 
                    min_value=0.1, 
                    max_value=0.5, 
                    step=0.05
                )
            )
        )
    # Dense layer
    if hp.Boolean("dense"):
        model.add(
            Dense(
                hp.Int(min_value=16, max_value=320, step=16, name="dense_dim"),
                activation="relu"
            )
        )
    # output layer
    model.add(
        Dense(1, activation='relu')
    )
    # optimizer
    optz = Adam(
        learning_rate=hp.Float(
            min_value=0.0005, max_value=0.01, step=0.0005, name="learning_rate"
        )
    )
    # compile model
    model.compile(
        optimizer=optz, 
        loss='mape', 
        metrics=["mae", "msle", "mse", "mape", RootMeanSquaredError()]
    )
    return model

In [14]:
tuner = keras_tuner.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=30,
    directory="../experiments",
    project_name="hyperparameter_tuning"
)

In [15]:
# train, val, _ = _build_centralized_model_dataset(Path("data/datasets/4func_10nodes/0"), config)

In [16]:
from azurefunctions_utils import load_dataset

train, val, _ = load_dataset(
  "../experiments/azurefunctions-dataset2019/10n_k3_15min/seed4850/0", 
  "centralized"
)

In [17]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=config.training.patience,
    min_delta=config.training.min_delta,
    restore_best_weights=True
)

tuner.search(
    train[0], 
    train[1], 
    validation_data=val, 
    callbacks=[early_stopping], 
    epochs=100
)

Trial 30 Complete [00h 00m 15s]
val_loss: 74.92354583740234

Best val_loss So Far: 74.92232513427734
Total elapsed time: 00h 06m 23s


In [18]:
tuner.results_summary(num_trials=1)

Results summary
Results in ../experiments/hyperparameter_tuning
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 04 summary
Hyperparameters:
lstm_layer_1_dim: 176
dropout_1: False
lstm_2: True
dropout_2: True
dense: False
learning_rate: 0.0055
lstm_layer_2_dim: 48
dense_dim: 48
dropout_2_rate: 0.35
Score: 74.92232513427734
